In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [52]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import os
from tqdm import tqdm
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
import cv2
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import Model
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, InputLayer, BatchNormalization, GlobalMaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img ,img_to_array
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


In [5]:
#Define Train-val-test Directories
#Covid_dir = os.path.join('/content/drive/MyDrive/COVID_CL',"CT_COVID")
#NonCOVID_dir = os.path.join('/content/drive/MyDrive/COVID_CL', "CT_NonCOVID")


In [6]:
#Defining levels of categories and encoding
levels = ["CT_COVID","CT_NonCOVID"]
levels_enc = {'CT_COVID': 1, 'CT_NonCOVID': 0}

In [29]:
Features = []
Labels = []

def gen_data(label,directory):
#     base = '/content/drive/MyDrive/COVID_CL'
    for scan in tqdm(os.listdir(directory)):
        label= label
        full_path = os.path.join(directory,scan)
        scan = cv2.imread(full_path,cv2.IMREAD_COLOR)
        scan = cv2.resize(scan, (224,224))
        
        Features.append(np.array(scan))
        Labels.append(str(label))

In [30]:
# Generating Data for training with labels.
gen_data(levels_enc.get('CT_COVID'), 'CT_COVID')
gen_data(levels_enc.get('CT_NonCOVID'), 'CT_NonCOVID')

100%|███████████████████████████████████████████████████████████████████████████████| 397/397 [00:00<00:00, 470.94it/s]


In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(np.array(Features), np.array(Labels), test_size=0.1, random_state=7)
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [85]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(671, 224, 224, 3)
(671, 2)
(75, 224, 224, 3)
(75, 2)


In [48]:
#Defining dense model
def dense_model():
    
    denseNet_model = DenseNet201(input_shape=(224,224,3) , weights='imagenet', include_top=False)
    denseNet_model.trainable = False
    
    flatten = Flatten()(denseNet_model.layers[-1].output)
    fc = Dense(units=512, activation='relu')(flatten)
    dropout = Dropout(0.35)(fc)
    output = Dense(2, activation='softmax')(dropout)
   
    model = Model(inputs=denseNet_model.input, outputs=output)
    
    model.summary()
    
    return model

#Assigning the dense model
dense_model= dense_model()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [89]:
#Defining learning parameters
starter_learning_rate = 1e-2
end_learning_rate = 1e-6
decay_steps = 10000
learning_rate = optimizers.schedules.PolynomialDecay(starter_learning_rate,decay_steps,end_learning_rate,power=0.4)

# Defining Optimizer, Loss & Metrics
opt = optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy()
met = 'accuracy'

# Compile the Model
dense_model.compile(optimizer=opt, loss=loss, metrics=[met])

In [92]:
#Augmentation on the train data
aug = ImageDataGenerator( shear_range=0.4, zoom_range=0.3, horizontal_flip=True, vertical_flip=True, fill_mode='nearest')


In [93]:


history = dense_model.fit_generator(aug.flow(X_train, y_train), validation_data=(X_test,y_test), epochs=10,
                    steps_per_epoch=len(X_train)//32, 
                    verbose=1)

# print('\n*** Fit is over ***')
# model.save("/content/drive/MyDrive/covid_cl.h5")
#model.save_weights("my_model.h5")

Epoch 1/10
20/20 [==============================] - 89s 4s/step - loss: 961.5543 - accuracy: 0.5544 - val_loss: 164.7436 - val_accuracy: 0.6533
Epoch 2/10
20/20 [==============================] - 70s 4s/step - loss: 151.1498 - accuracy: 0.6394 - val_loss: 35.8613 - val_accuracy: 0.5600
Epoch 3/10
20/20 [==============================] - 73s 4s/step - loss: 19.9142 - accuracy: 0.5855 - val_loss: 1.8120 - val_accuracy: 0.7333
Epoch 4/10
20/20 [==============================] - 77s 4s/step - loss: 1.8567 - accuracy: 0.6454 - val_loss: 0.9065 - val_accuracy: 0.6267
Epoch 5/10
20/20 [==============================] - 70s 4s/step - loss: 0.7263 - accuracy: 0.6339 - val_loss: 0.5685 - val_accuracy: 0.7333
Epoch 6/10
20/20 [==============================] - 70s 4s/step - loss: 0.6093 - accuracy: 0.6591 - val_loss: 0.5309 - val_accuracy: 0.7333
Epoch 7/10
20/20 [==============================] - 71s 4s/step - loss: 0.7059 - accuracy: 0.6517 - val_loss: 0.5475 - val_accuracy: 0.7467
Epoch 8/10
2

In [95]:
dense_model.evaluate(X_test,y_test)

3/3 [==============================] - 7s 2s/step - loss: 0.4707 - accuracy: 0.8133


[0.47068551182746887, 0.8133333325386047]

In [97]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D,Conv2D,Dense,BatchNormalization,concatenate,Input,Dropout,Maximum,Activation,Dense,Flatten,UpSampling2D,Conv2DTranspose
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.initializers as initializers
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import regularizers
from tensorflow.keras import Input

In [98]:
## VGG 16

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model

img_input = Input(shape=(224, 224, 3))

In [99]:
model = VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=img_input,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [100]:
last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(64, activation='relu', name='fc2')(x)
out = Dense(2, activation='sigmoid', name='output')(x)  ## 2 classes
model = Model(img_input, out)

for layer in model.layers[:-3]:
	layer.trainable = False

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [101]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [102]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [103]:
history = model.fit(X_train, y_train,
                              batch_size=16,
                              epochs=20, 
                              validation_data=(X_test, y_test),
                              callbacks=my_callbacks)

Epoch 1/20
42/42 [==============================] - 71s 2s/step - loss: 6.4319 - acc: 0.6817 - val_loss: 2.6597 - val_acc: 0.8000
Epoch 2/20
42/42 [==============================] - 77s 2s/step - loss: 0.9538 - acc: 0.8981 - val_loss: 1.9047 - val_acc: 0.8533
Epoch 3/20
42/42 [==============================] - 80s 2s/step - loss: 0.1513 - acc: 0.9845 - val_loss: 1.9224 - val_acc: 0.8667
Epoch 4/20
42/42 [==============================] - 74s 2s/step - loss: 0.0418 - acc: 0.9904 - val_loss: 1.7778 - val_acc: 0.8800
Epoch 5/20
42/42 [==============================] - 73s 2s/step - loss: 0.0303 - acc: 0.9935 - val_loss: 1.6857 - val_acc: 0.8800
Epoch 6/20
42/42 [==============================] - 73s 2s/step - loss: 0.0256 - acc: 0.9956 - val_loss: 2.2524 - val_acc: 0.8533
Epoch 7/20
42/42 [==============================] - 73s 2s/step - loss: 0.0405 - acc: 0.9970 - val_loss: 1.6930 - val_acc: 0.8800
Epoch 8/20
42/42 [==============================] - 74s 2s/step - loss: 0.0636 - acc: 1.00

In [106]:


## Test Accuracy

accuracy = history.history['acc']
validation_accuracy = history.history['val_acc']

model_pred = model.predict(X_test)
pred_set = model_pred > 0.5
test_acc = accuracy_score(y_test, pred_set)

precision, recall, f1score, _ = precision_recall_fscore_support(y_test, pred_set)

auc = roc_auc_score(y_test, pred_set)

print('Train Accuracy is', accuracy[-1])
print('Validation Accuracy obtained is' , validation_accuracy[-1])
print('Test Accuracy for the taken split is', test_acc)
print('Precision is', precision)
print('Recall', recall)
print('F1 Score', f1score)
print('AUC', auc)

Train Accuracy:	 0.9910581111907959
Val Accuracy:	 0.8399999737739563
Test Accuracy:	 0.8533333333333334
Precision:	 [0.84       0.88888889]
Recall:		 [0.93333333 0.8       ]
F1 Score:	 [0.88421053 0.84210526]
AUC:		 0.8500000000000001
